In [1]:
!pip install chromadb


In [1]:
import os

os.environ["GROQ_API_KEY"] = "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"


In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    # GROQ_API_KEY="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",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)



In [3]:
text=input()
respo=llm.invoke(text)
print(respo)

content="It seems like you didn't type anything. If you have a question or need help with something, feel free to ask, and I'll do my best to assist you." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 35, 'total_tokens': 71, 'completion_time': 0.144, 'prompt_time': 0.006529696, 'queue_time': 0.30776662600000004, 'total_time': 0.150529696}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_1c3737e28c', 'finish_reason': 'stop', 'logprobs': None} id='run-b5f2baf8-d1a9-42cf-9e90-6d7f85a63b97-0' usage_metadata={'input_tokens': 35, 'output_tokens': 36, 'total_tokens': 71}


In [4]:
import chromadb



In [5]:
client = chromadb.Client()

In [6]:
collection=client.create_collection(name="my_collection_1")

In [7]:
collection.add(
    documents=[
        "this document is about new york",
        "this document is about delhi"
    ]
    ,ids=['1','2']
)

In [8]:
all_doc=collection.get()
all_doc

{'ids': ['1', '2'],
 'embeddings': None,
 'documents': ['this document is about new york',
  'this document is about delhi'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [9]:
collection.query(query_texts='query is about chhole bhature')

Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2


{'ids': [['2', '1']],
 'embeddings': None,
 'documents': [['this document is about delhi',
   'this document is about new york']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.5588479042053223, 1.8114912509918213]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [10]:
collection.query(query_texts='query is about brooklyn bridge',n_results=2)

{'ids': [['1', '2']],
 'embeddings': None,
 'documents': [['this document is about new york',
   'this document is about delhi']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.095342755317688, 1.571172833442688]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [11]:
from langchain_community.document_loaders import WebBaseLoader

In [12]:
loader=WebBaseLoader("https://careers.ey.com/ey/job/Bengaluru-CA-Qualified-GL-Senior-associate-KA-560016/1090863301/")

In [13]:
page_data=loader.load().pop().page_content

In [14]:
print(page_data)














CA Qualified GL Senior associate Job Details | EY






























Cookie information
Welcome to the EY careers job search site. This website is based on the SuccessFactors software provided by SAP. On this page, functional and optional cookies are used to improve your experience and design our careers site more user-friendly and in line with your needs. In this context, cookies from providers in third countries may also be used and data may be transmitted to providers such as social media services outside the EU. For this we require your consent. By clicking "Accept All Cookies", you agree to these. This also includes your consent to the transmission of certain personal data to third countries, including the USA, in accordance with Art. 49 (1) (a) GDPR. You can select your settings by clicking “Modify Cookie Preferences” to confirm your choices from the optional cookie tracking and selecting the required cookies required to remain on the site. You can change 

In [15]:
from langchain_core.prompts import PromptTemplate

In [16]:
prompt_extract=PromptTemplate.from_template(
    """
    ### scraped text from website:
    {page_data}
    ### instruction:
    the scraped text is from the career's page of website. 
    your job is to extract the job postings and return the in json format containing 
    following keys: 'role','experience','skills',and 'description'.
    only return the valid json.
    ### valid json (no preamble):
    """
)

In [17]:
chain= prompt_extract | llm

In [18]:
res=chain.invoke(input={"page_data":page_data})

In [19]:
print(res.content)

[
  {
    "role": "CA Qualified GL Senior associate",
    "experience": "2-4 years",
    "skills": [
      "Good understanding of Indian Accounting Standards",
      "Working in multinational environment with different time zones",
      "High accuracy and on time delivery",
      "Succinct and structured oral and written communications skills",
      "Expertise in MS Excel, PowerPoint and handling large amount of data sets and tasks"
    ],
    "description": "The primary responsibility will be overseeing the month-end close, accruals, Audits, intercompany reconciliations, Schedules and related reporting requirement for India entities. You will work closely with other members of the Finance and accounting teams (India and Overseas) and provides accurate and timely information for month-end close. You will also be responsible to liaise and drive the month-end closure by interacting with various stakeholders (India and Overseas) to provide timely and qualitative inputs."
  }
]


In [20]:
import time
import random

retries = 5
for attempt in range(retries):
    try:
        res = chain.invoke(input={"page_data": page_data})
        break
    except Exception as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        if attempt < retries - 1:
            time.sleep(2 ** attempt + random.uniform(0, 1))
        else:
            raise


In [21]:
from langchain_core.output_parsers import JsonOutputParser

In [22]:
json_parser=JsonOutputParser()

In [23]:
json_res=json_parser.parse(res.content)

In [24]:
json_res

[{'role': 'CA Qualified GL Senior associate',
  'experience': '2-4 years',
  'skills': ['Good understanding of Indian Accounting Standards',
   'Working in multinational environment with different time zones',
   'High accuracy and on time delivery',
   'Succinct and structured oral and written communications skills',
   'Expertise in MS Excel, PowerPoint and handling large amount of data sets and tasks'],
  'description': 'The primary responsibility will be overseeing the month-end close, accruals, Audits, intercompany reconciliations, Schedules and related reporting requirement for India entities. You will work closely with other members of the Finance and accounting teams (India and Overseas) and provides accurate and timely information for month-end close. You will also be responsible to liaise and drive the month-end closure by interacting with various stakeholders (India and Overseas) to provide timely and qualitative inputs'}]

In [25]:
type(json_res)

list

In [26]:

from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [27]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)


In [28]:
json_res

[{'role': 'CA Qualified GL Senior associate',
  'experience': '2-4 years',
  'skills': ['Good understanding of Indian Accounting Standards',
   'Working in multinational environment with different time zones',
   'High accuracy and on time delivery',
   'Succinct and structured oral and written communications skills',
   'Expertise in MS Excel, PowerPoint and handling large amount of data sets and tasks'],
  'description': 'The primary responsibility will be overseeing the month-end close, accruals, Audits, intercompany reconciliations, Schedules and related reporting requirement for India entities. You will work closely with other members of the Finance and accounting teams (India and Overseas) and provides accurate and timely information for month-end close.'}]

In [29]:
job=json_res[0]['skills']

In [30]:
import pandas as pd

In [31]:
df=pd.read_csv(r"C:\Users\ASUS\Downloads\my_portfolio.csv")

In [32]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [33]:
print(1+"hi")

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [38]:
import uuid

In [39]:
for _, row in df.iterrows():
    print(row['Links'])

https://example.com/react-portfolio
https://example.com/angular-portfolio
https://example.com/vue-portfolio
https://example.com/python-portfolio
https://example.com/java-portfolio
https://example.com/flutter-portfolio
https://example.com/wordpress-portfolio
https://example.com/magento-portfolio
https://example.com/react-native-portfolio
https://example.com/ios-portfolio
https://example.com/android-portfolio
https://example.com/kotlin-android-portfolio
https://example.com/android-tv-portfolio
https://example.com/ios-ar-portfolio
https://example.com/xamarin-portfolio
https://example.com/kotlin-backend-portfolio
https://example.com/typescript-frontend-portfolio
https://example.com/full-stack-js-portfolio
https://example.com/ml-python-portfolio
https://example.com/devops-portfolio


In [40]:
client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        print(_,"----",row)
        collection.add(documents=[row['Techstack']],metadatas=[{'links':row['Links']}],ids=[str(uuid.uuid4())])

In [41]:
links =collection.query(query_texts=["Experince in python ","Experties in react native "],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [43]:
prompt_email=PromptTemplate.from_template(
    """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """

)

In [50]:
prompt_email=PromptTemplate.from_template(
    """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """

)
chain_email= prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Accounting and Data Management Solutions for Your Business

Dear [Client's Name],

I came across your job description for an accounting professional and was impressed by the emphasis on expertise in Indian Accounting Standards, data handling, and effective communication. As a Business Development Executive at AtliQ, I'd like to introduce you to our company's capabilities in providing tailored solutions that cater to your business needs.

At AtliQ, we specialize in developing AI-powered and software solutions that streamline business processes, enhance scalability, and optimize costs. Our team of experts has extensive experience in handling large datasets and creating customized tools to improve data management and analysis.

Our expertise in MS Excel, PowerPoint, and data handling aligns perfectly with your requirements. We've developed solutions for various clients, including those in the finance and accounting sectors, and have a proven track record of delivering high

In [59]:
chain_email= prompt_email | llm
res=chain_email.invoke({"job_description":job[0]['skills'],"link_list":links})
print(res.content)

KeyError: "Input to PromptTemplate is missing variables {'links_list'}.  Expected: ['links_list'] Received: ['job_description', 'link_list']\nNote: if you intended {links_list} to be part of the string and not a variable, please escape it with double curly braces like: '{{links_list}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [58]:
job=json_res
job[0]['skills']

['Good understanding of Indian Accounting Standards',
 'Working in multinational environment with different time zones',
 'High accuracy and on time delivery',
 'Succinct and structured oral and written communications skills',
 'Expertise in MS Excel, PowerPoint and handling large amount of data sets and tasks']